In [1]:
from scipy       import integrate
import numpy             as np

In [1]:
class Peak():
	def __init__(self, path_to_input, dst_dir):
		self.filename = path_to_input
		self.dst_dir = dst_dir
		self.wavenumber = []
		self.intensity = []
		self.normalized = []
		self.truncated = []
		self.wavenumber2 = []
		self.concentration_m = []
		return
	def run(self):
		self.parse()
		self.normalize()
		self.truncate()
		self.write_results()
		self.area()
		return
	def parse(self):
		with open(self.filename, encoding="utf8", errors="ignore") as f:
			for line in f:
				if line.startswith('#'):
					continue
				parts = line.split()
				w = float(parts[0])
				i = float(parts[1])
				self.wavenumber.append(w)
				self.intensity.append(i)
		return 
	def normalize(self):
		index = 0
		for ind, val in enumerate(self.wavenumber):
			if val > 1400:
				index = ind
		m = max(self.intensity[:index])
		for i in self.intensity:
			self.normalized.append(i/m)
		return
	def truncate(self):
		index1 = 0
		index2 = 0
		for ind1, val1 in enumerate(self.wavenumber):
			if val1 < 762 and val1 > 758:
				index1 = ind1
			if val1 < 682 and val1 > 678:
				index2 = ind1
			self.truncated = self.normalized[index1:index2]
			self.wavenumber2 = self.wavenumber[index1:index2]
		return
	def area(self):
			area_peak = integrate.simps(self.truncated, self.wavenumber2)
			a = -20.46985
			b = -0.6228
			self.concentration_m = (area_peak-b)/a
			return
	def plotting(self):
		plt.plot(self.wavenumber2, self.truncated)
		plt.fill_between(self.wavenumber2, self.truncated, color = "grey",alpha = 0.3, hatch = '|')
		plt.show()
		return 
	def write_results(self):#存好700附近的标准化数据
		with open(self.dst_dir+'/'+self.filename[self.filename.rfind('/'):-4]+"_700PEAK.txt", 'w', encoding="utf8", errors="ignore") as f:
			for i in range(len(self.truncated)):
				f.write(str(self.wavenumber2[i]))
				f.write('\t')
				f.write(str(self.truncated[i]))
				f.write('\n')
		return